In [79]:
import pandas as pd
import datetime
import numpy as np
import pickle
import os
import pymongo
import requests
import json
from datetime import date
from datetime import timedelta 

## LOADING DATA

In [80]:
concats = pd.DataFrame()
for i in range(14,20):
    concats = pd.concat([concats , pd.read_csv("data/"+str(i)+"1.csv")], sort = False,ignore_index=True)
    
for i in range(14,20):
    concats = pd.concat([concats , pd.read_csv("data/"+str(i)+"2.csv")], sort = False,ignore_index=True)

## Data engineering

In [81]:
def stringToDate(string):
    string = string.split("/")
    if(len(string[2]) == 4):
        string[2] = string[2][2:]
    date = string[0]+"/"+ string[1]+"/20"+string[2]
    date = datetime.datetime.strptime(date, '%d/%m/%Y')
    return date

In [82]:
def hash_match(date, away, home, ver):
    date = date.split("/")
    return hash(date[0]+date[1]+date[2]+away+home+str(ver))    

In [83]:
def getLastMatchId(id_match, nb_earlier_match):
    dates = concats['Date'].apply(stringToDate)
    #homepart
    ids_HH = dates[(dates < dates[id_match]) & (concats["HomeTeam"] == concats["HomeTeam"][id_match])].index
    ids_HA = dates[(dates < dates[id_match]) & (concats["AwayTeam"] == concats["HomeTeam"][id_match])].index
    ids_home = np.array(ids_HH,ids_HA)
    if(ids_home.shape[0] < nb_earlier_match):
        return np.nan
    else:
        ids_home = ids_home[ids_home.argsort()[-nb_earlier_match:][::-1]]
    #awaypart
    
    ids_AH = dates[(dates < dates[id_match]) & (concats["HomeTeam"] == concats["AwayTeam"][id_match])].index
    ids_AA = dates[(dates < dates[id_match]) & (concats["AwayTeam"] == concats["AwayTeam"][id_match])].index
    ids_away = np.array(ids_AH,ids_AA)
    if(ids_away.shape[0] < nb_earlier_match):
        return np.nan
    else:
        ids_away = ids_away[ids_away.argsort()[-nb_earlier_match:][::-1]]
    return np.array([ids_away, ids_home])

In [84]:
concats = concats[pd.notnull(concats['FTR'])]
concats = concats.drop(columns = concats.columns[23:])
concats['ID_FTR'] = concats.FTR.astype('category').cat.codes

In [85]:
concats = concats.drop(columns = concats.columns[9])

In [86]:
hashmap = {}

In [87]:
with open('hm.pickle', 'rb') as handle:
    hashmap = pickle.load(handle)

In [88]:
ids = []
for i in range(concats.shape[0]):
    vals = concats.iloc[i,[1,2,3]].values
    hashed = hash_match(vals[0],vals[1], vals[2], 1)
    if hashed in hashmap:
        ids.append(hashmap[hashed])
    else:
        id_match = getLastMatchId(concats.index[i],3)
        ids.append(id_match)
        hashmap[hashed] = id_match


In [89]:
with open('hm.pickle', 'wb') as handle:
    pickle.dump(hashmap, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [90]:
ids = pd.DataFrame(ids)
ids = ids.reindex(concats.index)
ids = ids.dropna()

In [91]:
concats = concats.fillna(0.0)

In [92]:
ids_m = pd.DataFrame()
ids_m["_id"] = concats.iloc[:,[1,2,3]].apply(lambda row: hash_match(row.iloc[0],row.iloc[1],row.iloc[2],1), axis = 1)

In [93]:
concats['AAG'] = np.nan
concats['AAS'] = np.nan
concats['AASO'] = np.nan
concats['AAF'] = np.nan
concats['AAC'] = np.nan
concats['AAY'] = np.nan
concats['AAR'] = np.nan

concats['HAG'] = np.nan
concats['HAS'] = np.nan
concats['HASO'] = np.nan
concats['HAF'] = np.nan
concats['HAC'] = np.nan
concats['HAY'] = np.nan
concats['HAR'] = np.nan

for i in ids.index:
    #awaypart
    ids_away = ids[0][i][0]
    avg_goals = 0
    avg_shots = 0
    avg_shots_on = 0
    avg_fouls = 0
    avg_corners = 0
    avg_yellow = 0
    avg_red = 0
    for k in ids_away:
        if(concats.loc[i,"AwayTeam"] == concats.loc[k,"AwayTeam"]):
            avg_goals += concats.loc[k,"FTAG"]
            avg_shots += concats.loc[k,"AS"]
            avg_shots_on += concats.loc[k,"AST"]
            avg_fouls += concats.loc[k,"AF"]
            avg_corners = concats.loc[k,"AC"]
            avg_yellow += concats.loc[k,"AY"]
            avg_red += concats.loc[k,"AR"]
        else:
            avg_goals += concats.loc[k,"FTHG"]
            avg_shots += concats.loc[k,"HS"]
            avg_shots_on += concats.loc[k,"HST"]
            avg_fouls += concats.loc[k,"HF"]
            avg_corners = concats.loc[k,"HC"]
            avg_yellow += concats.loc[k,"HY"]
            avg_red += concats.loc[k,"HR"]
            
    concats.loc[i,'AAG'] = avg_goals
    concats.loc[i,'AAS'] = avg_shots
    concats.loc[i,'AASO'] = avg_shots_on
    concats.loc[i,'AAF'] = avg_fouls
    concats.loc[i,'AAC'] = avg_corners
    concats.loc[i,'AAY'] = avg_yellow
    concats.loc[i,'AAR'] = avg_red

            
    ids_home = ids[0][i][1]
    avg_goals = 0
    avg_shots = 0
    avg_shots_on = 0
    avg_fouls = 0
    avg_corners = 0
    avg_yellow = 0
    avg_red = 0
    for k in ids_home:
        if(concats.loc[i,"HomeTeam"] == concats.loc[k,"AwayTeam"]):
            avg_goals += concats.loc[k,"FTAG"]
            avg_shots += concats.loc[k,"AS"]
            avg_shots_on += concats.loc[k,"AST"]
            avg_fouls += concats.loc[k,"AF"]
            avg_corners = concats.loc[k,"AC"]
            avg_yellow += concats.loc[k,"AY"]
            avg_red += concats.loc[k,"AR"]
        else:
            avg_goals += concats.loc[k,"FTHG"]
            avg_shots += concats.loc[k,"HS"]
            avg_shots_on += concats.loc[k,"HST"]
            avg_fouls += concats.loc[k,"HF"]
            avg_corners = concats.loc[k,"HC"]
            avg_yellow += concats.loc[k,"HY"]
            avg_red += concats.loc[k,"HR"]
        
       
    concats.loc[i,'HAG'] = avg_goals
    concats.loc[i,'HAS'] = avg_shots
    concats.loc[i,'HASO'] = avg_shots_on
    concats.loc[i,'HAF'] = avg_fouls
    concats.loc[i,'HAC'] = avg_corners
    concats.loc[i,'HAY'] = avg_yellow
    concats.loc[i,'HAR'] = avg_red

In [94]:
concats = concats.drop([x for x in concats.index.values if x not in ids.index.values])

In [95]:
concats.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY',
       'HR', 'AR', 'B365H', 'ID_FTR', 'AAG', 'AAS', 'AASO', 'AAF', 'AAC',
       'AAY', 'AAR', 'HAG', 'HAS', 'HASO', 'HAF', 'HAC', 'HAY', 'HAR'],
      dtype='object')

In [96]:
concats = concats.drop(columns = concats.columns[4:6])
concats = concats.drop(columns = concats.columns[5:20])

In [97]:
normalized_df=(concats[concats.columns[6:]]-concats[concats.columns[6:]].mean())/concats[concats.columns[6:]].std()

In [98]:
concats[concats.columns[6:]] = normalized_df

In [99]:
concats.columns[6:13]

Index(['AAG', 'AAS', 'AASO', 'AAF', 'AAC', 'AAY', 'AAR'], dtype='object')

In [100]:
concats.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTR', 'ID_FTR', 'AAG', 'AAS',
       'AASO', 'AAF', 'AAC', 'AAY', 'AAR', 'HAG', 'HAS', 'HASO', 'HAF', 'HAC',
       'HAY', 'HAR'],
      dtype='object')

In [101]:
concats.columns[13:]

Index(['HAG', 'HAS', 'HASO', 'HAF', 'HAC', 'HAY', 'HAR'], dtype='object')

In [154]:
ids_team = list()
names = concats["HomeTeam"].drop_duplicates()
for i in names:
    df = np.array(concats[concats["HomeTeam"] == i].index, concats[concats["AwayTeam"] == i].index)
    ids_team.append(max(df))

values = list()
for i in range(len(names)):
    if(concats.loc[ids_team[i], "HomeTeam"] == names.iloc[i]):
        values.append(concats.loc[ids_team[i], concats.columns[13:]].values.astype(float))
    else:
        values.append(concats.loc[ids_team[i], concats.columns[6:13]].values.astype(float))
        
team_data = pd.DataFrame(values)
team_data = team_data.rename(columns = {0:"a",1:"b",2:"c",3:"d",4:"e",5:"f",6:"g",7:"h"})
team_data.insert(0, '_id',  names.values ) 

In [118]:
concats.insert(0, '_id',  ids_m["_id"])

ValueError: cannot insert _id, already exists

In [120]:
teams = pd.DataFrame(concats["HomeTeam"].drop_duplicates())
teams = teams.rename(columns = {"HomeTeam" : "football.co.uk"})

In [121]:
teams["football.co.uk"] = teams["football.co.uk"].sort_values()

teams["id"] = teams["football.co.uk"].apply(lambda x : x.replace(" ", "_" ).lower())

In [122]:
teams.to_csv("team_matching.csv")

In [161]:
def update_db(data, coll):
    for index, row in data.iterrows():
        coll.replace_one({'_id':row['_id']}, row.to_dict(), True)

In [124]:
def fetch_old(name_post):
    client = pymongo.MongoClient("mongodb://192.168.99.100:27017")
    db = client["plp"][name_post]
    a= list(db["match"].find({}))
    a = pd.DataFrame(a)
    return a

In [125]:
def remove_match_db(id):
    query = { "_id": id}
    coll.delete_one(query)    

In [159]:
def pandas_to_mongo(df, name_post):
    client = pymongo.MongoClient("mongodb://192.168.99.100:27017")
    coll = client["plp"][name_post]
    old = fetch_old(name_post)
    new = df

    if len(old)==0:
        old = new
    update_filtered = new[new["_id"].isin(old["_id"])]

    update_db(update_filtered,coll)
    
    print("Done.")
    print("Updated : "+str(len(update_filtered)))
    print("Total matches in base : "+str(coll.estimated_document_count()))
        

In [127]:
concats

,_id,Div,Date,HomeTeam,AwayTeam,FTR,ID_FTR,AAG,AAS,AASO,...,AAC,AAY,AAR,HAG,HAS,HASO,HAF,HAC,HAY,HAR
54,4041509492721073641,F1,20/09/14,Nantes,Nice,H,2,0.262873,0.895837,0.375430,...,2.587834,-0.089255,-0.435110,-1.035393,0.953032,0.636636,0.907359,-0.103268,-0.417868,-0.429740
57,-3253533984583305966,F1,21/09/14,Lille,Montpellier,D,1,-0.604166,0.692484,-0.006526,...,0.493973,-0.089255,-0.435110,-0.172064,0.800350,0.636636,0.907359,1.388582,-0.417868,-0.429740
60,8346812620433998539,F1,23/09/14,Reims,Marseille,A,0,1.129912,1.709250,1.393978,...,0.793096,-0.089255,-0.435110,-0.172064,0.138727,0.126082,0.608271,-0.700008,-0.755575,-0.429740
62,9210601909339019084,F1,24/09/14,Bastia,Nantes,D,1,-0.604166,1.200867,0.630067,...,0.493973,-0.426711,-0.435110,0.259600,0.291409,0.253721,1.455686,-0.700008,1.270663,-0.429740
64,-1923595744662626081,F1,24/09/14,Evian Thonon Gaillard,Lens,H,2,-0.170646,0.794161,0.630067,...,1.092219,1.598025,-0.435110,-1.467057,0.240515,-0.512110,1.106751,0.493472,0.595250,-0.429740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4194,-5604791961990090543,F2,10/01/2020,Le Mans,Auxerre,A,0,0.696392,0.692484,0.757385,...,-0.702518,2.272936,1.584183,-0.172064,1.207502,1.530106,1.156598,1.090212,-0.080162,-0.429740
4195,-8698635011057970952,F2,10/01/2020,Nancy,Valenciennes,H,2,1.129912,0.844999,0.630067,...,1.092219,0.923113,-0.435110,-1.035393,0.342303,-0.129195,1.256294,0.493472,0.932957,-0.429740
4196,-5880345821002083533,F2,10/01/2020,Niort,Le Havre,A,0,-0.604166,0.844999,0.502748,...,-0.104273,0.585657,-0.435110,-0.172064,0.036939,0.253721,1.156598,-0.401638,1.608369,-0.429740
4197,-1831289101984127392,F2,10/01/2020,Rodez,Chateauroux,A,0,-1.037685,0.336616,0.120793,...,-0.702518,2.272936,1.584183,-0.603728,-0.013955,-0.001556,1.007055,0.195102,0.595250,1.626359


In [162]:
pandas_to_mongo(team_data,"team")
pandas_to_mongo(concats,"match")

Done.
Updated : 51
Total matches in base : 51
Done.
Updated : 4019
Total matches in base : 4019


In [163]:
def get_team_features(team_name):
    client = pymongo.MongoClient("mongodb://192.168.99.100:27017")
    db = client['plp']
    dic = db["team"].find_one({"_id": {'$regex':'^'+team_name}})
    del dic['_id']
    return np.fromiter(dic.values(),dtype = float)